In [2]:
import yfinance as yf
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.optimize import minimize

In [4]:
# Download historical data for an ETF (e.g., SPY)
names = ["IUIT.L","RBOT.L", "IUFS.L","HEAL.L","VUAA.L", "DGTL.L", "ISAC.L", "DFEN.DE", "MEUD.PA", "ESIF.DE","VVSM.DE", "XAIX.DE" ]
daily_returns = pd.DataFrame()
for ticker in names:
    ticker_data = yf.download(ticker, start="2015-01-01", end="2025-11-03")['Close'].pct_change()
    daily_returns[ticker] = ticker_data

C:\Users\Radek\AppData\Local\Temp\ipykernel_2860\2852733541.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  ticker_data = yf.download(ticker, start="2015-01-01", end="2025-11-03")['Close'].pct_change()
[*********************100%***********************]  1 of 1 completed
C:\Users\Radek\AppData\Local\Temp\ipykernel_2860\2852733541.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  ticker_data = yf.download(ticker, start="2015-01-01", end="2025-11-03")['Close'].pct_change()
[*********************100%***********************]  1 of 1 completed
C:\Users\Radek\AppData\Local\Temp\ipykernel_2860\2852733541.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  ticker_data = yf.download(ticker, start="2015-01-01", end="2025-11-03")['Close'].pct_change()
[*********************100%***********************]  1 of 1 completed
C:\Users\Radek\AppData\Local\Temp\ipykernel_2860\2852733541.py:5: FutureW

In [5]:
# Drop 'DFEN.DE' and 'MEUD.PA' columns
daily_returns_filtered = daily_returns.drop(columns=['DFEN.DE', 'MEUD.PA'])
daily_returns_filtered = daily_returns_filtered.dropna()

# Calculate the correlation matrix
corr_matrix = daily_returns_filtered.corr()

# Calculate the covariance matrix
cov_matrix = daily_returns_filtered.cov()

# Visualize the correlation matrix
# plt.figure(figsize=(10, 8))
# sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm")
# plt.title("Correlation Matrix of Selected Tickers")
# plt.xlabel("Tickers")
# plt.ylabel("Tickers")
# plt.show()

# Visualize the covariance matrix
# plt.figure(figsize=(10, 8))
# sns.heatmap(cov_matrix, annot=True, fmt=".2e", cmap="coolwarm")
# plt.title("Covariance Matrix of Selected Tickers")
# plt.xlabel("Tickers")
# plt.ylabel("Tickers")
# plt.show()

#corr_matrix
#cov_matrix


In [6]:
average_daily_returns = daily_returns_filtered.mean()
volatility_array = np.array([cov_matrix[i][i]for i in cov_matrix.columns])

In [7]:
results = pd.DataFrame({
    'average_daily_returns': average_daily_returns,
    'volatility': volatility_array,
    'Initial share': np.repeat(1 / len(average_daily_returns), len(average_daily_returns))
})

total_volatility = np.sqrt(np.dot(results['Initial share'].values, np.dot(cov_matrix.values, results['Initial share'].values.T)))
print(total_volatility)
print(results)



0.011527409176998358
         average_daily_returns  volatility  Initial share
IUIT.L                0.000920    0.000215            0.1
RBOT.L                0.000421    0.000216            0.1
IUFS.L                0.000614    0.000152            0.1
HEAL.L               -0.000051    0.000165            0.1
VUAA.L                0.000597    0.000104            0.1
DGTL.L                0.000193    0.000192            0.1
ISAC.L                0.000480    0.000096            0.1
ESIF.DE               0.000817    0.000141            0.1
VVSM.DE               0.001121    0.000364            0.1
XAIX.DE               0.000685    0.000177            0.1


In [8]:

def optimize_portfolio(results, cov_matrix):
    num_assets = len(results['average_daily_returns'])
    
    # Objective function: Minimize portfolio variance
    def portfolio_volatility(weights):
        return np.sqrt(weights @ cov_matrix @ weights.T)
    
    # Constraints: Weights sum to 1
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    
    # Bounds: Weights between 0 and 1
    bounds = tuple((0, 1) for _ in range(num_assets))
    
    # Initial guess: Equal distribution
    initial_guess = np.array(num_assets * [1. / num_assets])
    
    # Optimize
    # method='SLSQP' is Sequential Least Squares Programming

    result = minimize(portfolio_volatility, initial_guess, method='SLSQP', bounds=bounds, constraints=constraints)
    
    return result



In [9]:
optimize_portfolio(results, cov_matrix)

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.009782928141122737
       x: [ 0.000e+00  7.510e-17  0.000e+00  9.536e-02  2.595e-01
            1.647e-17  3.192e-01  2.672e-01  6.651e-18  5.871e-02]
     nit: 15
     jac: [ 1.233e-02  1.321e-02  1.029e-02  1.013e-02  9.702e-03
            1.221e-02  9.564e-03  9.924e-03  1.387e-02  1.013e-02]
    nfev: 165
    njev: 15

In [10]:
#testing
average_daily_returns_test = np.array([6,2,4])
volatility_array = np.array([8.94427191000,	4.47213595500,	8.94427191000])
results_test = pd.DataFrame({
    'average_daily_returns': average_daily_returns_test,
    'volatility': volatility_array,
    'Initial share': np.repeat(1 / len(average_daily_returns_test), len(average_daily_returns_test))
})
cov_matrix_test = pd.DataFrame({
    'asset1': np.array([0.00800, -0.00200, 0.00400]),
    'asset2': np.array([-0.00200, 0.00200, -0.00200]),
    'asset3': np.array([0.00400, -0.00200, 0.00800])
})
print(cov_matrix_test)
print(results_test)
print(optimize_portfolio(results_test, cov_matrix_test))

   asset1  asset2  asset3
0   0.008  -0.002   0.004
1  -0.002   0.002  -0.002
2   0.004  -0.002   0.008
   average_daily_returns  volatility  Initial share
0                      6    8.944272       0.333333
1                      2    4.472136       0.333333
2                      4    8.944272       0.333333
 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.025819888974717046
       x: [ 1.667e-01  6.667e-01  1.667e-01]
     nit: 4
     jac: [ 2.630e-02  2.558e-02  2.630e-02]
    nfev: 17
    njev: 4


In [25]:
risk_free_rate = 0.0001  # Example daily risk-free rate
results['excess_return'] = results['average_daily_returns'] - risk_free_rate
results['portfolio_allocation_normalized'] = results["excess_return"] @ np.linalg.inv(cov_matrix)
results


,average_daily_returns,volatility,Initial share,excess_return,sharpe_portfolio,sharpe_positions,portfolio_allocation_normalized
IUIT.L,0.000920,0.000215,0.1,0.000820,0.916653,5.824819,5.824819
RBOT.L,0.000421,0.000216,0.1,0.000321,-0.740352,-4.704524,-4.704524
IUFS.L,0.000614,0.000152,0.1,0.000514,-0.197476,-1.254849,-1.254849
HEAL.L,-0.000051,0.000165,0.1,-0.000151,-0.518105,-3.292266,-3.292266
VUAA.L,0.000597,0.000104,0.1,0.000497,2.072072,13.166857,13.166857
DGTL.L,0.000193,0.000192,0.1,0.000093,-1.199291,-7.620819,-7.620819
ISAC.L,0.000480,0.000096,0.1,0.000380,-0.905382,-5.753193,-5.753193
ESIF.DE,0.000817,0.000141,0.1,0.000717,1.118214,7.105625,7.105625
VVSM.DE,0.001121,0.000364,0.1,0.001021,0.186195,1.183163,1.183163
XAIX.DE,0.000685,0.000177,0.1,0.000585,0.267471,1.699626,1.699626
